In [5]:

import pandas as pd
import numpy as np
import re
from pathlib import Path

# Rutas a CSV 
files = {
    "altaverapaz": "datos_csv/altaverapaz.csv",
    "bajaverapaz": "datos_csv/bajaverapaz.csv",
    "chimaltenango": "datos_csv/chimaltenango.csv",
    "chiquimula": "datos_csv/chiquimula.csv",
    "ciudadcapital": "datos_csv/ciudadcapital.csv",
    "elprogreso": "datos_csv/elprogreso.csv",
    "escuintla": "datos_csv/escuintla.csv",
    "guatemala": "datos_csv/guatemala.csv",
    "izabal": "datos_csv/izabal.csv",
    "jalapa": "datos_csv/jalapa.csv",
    "jutiapa": "datos_csv/jutiapa.csv",
    "peten": "datos_csv/peten.csv",
    "quetzaltenango": "datos_csv/quetzaltenango.csv",
    "quiche": "datos_csv/quiche.csv",
    "retalhuleu": "datos_csv/retalhuleu.csv",
    "sacatepequez": "datos_csv/sacatepequez.csv",
    "sanmarcos": "datos_csv/sanmarcos.csv",
    "santarosa": "datos_csv/santarosa.csv",
    "solola": "datos_csv/solola.csv",
    "suchitipequez": "datos_csv/suchitipequez.csv",
    "totonicapan": "datos_csv/totonicapan.csv",
    "zacapa": "datos_csv/zacapa.csv",
}

# Carpeta de salida para los CSV limpios
out_dir = Path("salidas_limpias")
out_dir.mkdir(exist_ok=True, parents=True)
print("OK: entorno listo. Carpeta de salida:", out_dir.resolve())


OK: entorno listo. Carpeta de salida: C:\Users\Silvia\Documents\Cuarto Año\Segundo Semestre\Data\Proyecto1-DataScience-LimpiezaDeDatos\salidas_limpias


In [6]:
def load_and_fix_header(csv_path: str) -> pd.DataFrame:
    """
    Corrige el caso:
      - Fila 0: '0,1,2,...' (basura)
      - Fila 1: encabezado real ('CODIGO', 'DISTRITO', ...)
    Si no viene así, usa la primera fila como header.
    """
    raw = pd.read_csv(csv_path, header=None, dtype=str, keep_default_na=False)
    row0 = [str(x).strip() for x in raw.iloc[0].tolist()]
    row1 = [str(x).strip() for x in raw.iloc[1].tolist()] if len(raw) > 1 else None

    looks_like_numeric_header = all(re.fullmatch(r"\d+", x) for x in row0 if x != "")
    has_codigo_in_row1 = row1 is not None and any(x.upper() == "CODIGO" for x in row1)

    if looks_like_numeric_header and has_codigo_in_row1:
        header = row1
        df = raw.iloc[2:].copy()
    else:
        header = row0
        df = raw.iloc[1:].copy()

    df.columns = [str(c).strip() for c in header]
    for c in df.columns:
        df[c] = df[c].astype(str).str.strip()
    return df.reset_index(drop=True)

def standardize_colnames(cols):
    mapping = {
        "CODIGO":"CODIGO","DISTRITO":"DISTRITO","DEPARTAMENTO":"DEPARTAMENTO",
        "MUNICIPIO":"MUNICIPIO","ESTABLECIMIENTO":"ESTABLECIMIENTO","DIRECCION":"DIRECCION",
        "TELEFONO":"TELEFONO","SUPERVISOR":"SUPERVISOR","DIRECTOR":"DIRECTOR","NIVEL":"NIVEL",
        "SECTOR":"SECTOR","AREA":"AREA","STATUS":"STATUS","MODALIDAD":"MODALIDAD",
        "JORNADA":"JORNADA","PLAN":"PLAN","DEPARTAMENTAL":"DEPARTAMENTAL",
    }
    out = []
    for c in cols:
        key = str(c).strip().upper()
        out.append(mapping.get(key, key))
    return out


In [7]:

for name, path in files.items():
    df = load_and_fix_header(path)
    df.columns = standardize_colnames(df.columns)

    # Resumen básico
    n_filas, n_cols = df.shape
    nulos = {c: int((df[c] == '').sum()) for c in df.columns}
    duplicados_codigo = int(df['CODIGO'].duplicated().sum()) if 'CODIGO' in df.columns else "N/A"

    # Teléfonos
    if 'TELEFONO' in df.columns:
        tel_digits = df['TELEFONO'].astype(str).str.replace(r"\D", "", regex=True)
        tel_longitudes = tel_digits.str.len().value_counts().sort_index().to_dict()
    else:
        tel_longitudes = {}

    print(f"--- {name.upper()} ---")
    print(f"Forma: {n_filas} filas x {n_cols} columnas")
    print(f"Duplicados por CODIGO: {duplicados_codigo}")
    print(f"Vacíos por columna: {nulos}")
    print(f"Longitudes en TELEFONO: {tel_longitudes}")
    print()


--- ALTAVERAPAZ ---
Forma: 295 filas x 17 columnas
Duplicados por CODIGO: 0
Vacíos por columna: {'CODIGO': 1, 'DISTRITO': 1, 'DEPARTAMENTO': 1, 'MUNICIPIO': 1, 'ESTABLECIMIENTO': 1, 'DIRECCION': 1, 'TELEFONO': 7, 'SUPERVISOR': 1, 'DIRECTOR': 1, 'NIVEL': 1, 'SECTOR': 1, 'AREA': 1, 'STATUS': 1, 'MODALIDAD': 1, 'JORNADA': 1, 'PLAN': 1, 'DEPARTAMENTAL': 1}
Longitudes en TELEFONO: {0: 7, 7: 1, 8: 286, 16: 1}

--- BAJAVERAPAZ ---
Forma: 95 filas x 17 columnas
Duplicados por CODIGO: 0
Vacíos por columna: {'CODIGO': 1, 'DISTRITO': 1, 'DEPARTAMENTO': 1, 'MUNICIPIO': 1, 'ESTABLECIMIENTO': 1, 'DIRECCION': 1, 'TELEFONO': 1, 'SUPERVISOR': 1, 'DIRECTOR': 1, 'NIVEL': 1, 'SECTOR': 1, 'AREA': 1, 'STATUS': 1, 'MODALIDAD': 1, 'JORNADA': 1, 'PLAN': 1, 'DEPARTAMENTAL': 1}
Longitudes en TELEFONO: {0: 1, 8: 92, 16: 2}

--- CHIMALTENANGO ---
Forma: 305 filas x 17 columnas
Duplicados por CODIGO: 0
Vacíos por columna: {'CODIGO': 1, 'DISTRITO': 1, 'DEPARTAMENTO': 1, 'MUNICIPIO': 1, 'ESTABLECIMIENTO': 1, 'DIRECCI

In [8]:
import pandas as pd
import re

# Limpiar y extraer solo dígitos del campo TELEFONO
tel_digits = df['TELEFONO'].fillna("").astype(str).str.replace(r"\D", "", regex=True)

# Teléfonos válidos deben tener exactamente 8 dígitos
# Y empezar con dígitos válidos para Guatemala: 2, 3, 4, 5, 6 o 7
valid_start_digits = ("2", "3", "4", "5", "6", "7")

# Crear máscara de teléfonos inválidos
invalid_mask = (
    (tel_digits.str.len() != 8) |                       # longitud distinta de 8
    (~tel_digits.str.startswith(valid_start_digits))    # o no empieza con dígito válido
)

# Filtrar los registros con problemas
df_invalid_tel = df.loc[invalid_mask, ["CODIGO", "ESTABLECIMIENTO", "DIRECCION", "TELEFONO"]]

# Mostrar resultados
print("Teléfonos inválidos encontrados:", len(df_invalid_tel))
display(df_invalid_tel.head(10))  # muestra los primeros 10 registros inválidos


# Textos con comillas dobles o espacios duplicados en ESTABLECIMIENTO y DIRECCION
weird_name_mask = df["ESTABLECIMIENTO"].str.contains(r"[\"'”“]{1}|  ", regex=True, na=False)
weird_dir_mask = df["DIRECCION"].str.contains(r"[\"'”“]{1}|  ", regex=True, na=False)

print("Establecimientos con comillas/espacios dobles:", weird_name_mask.sum())
display(df.loc[weird_name_mask, ["CODIGO", "ESTABLECIMIENTO"]].head(10))

print("Direcciones con comillas/espacios dobles:", weird_dir_mask.sum())
display(df.loc[weird_dir_mask, ["CODIGO", "DIRECCION"]].head(10))

# Posibles duplicados (mismo municipio + establecimiento + dirección, normalizados)
def normalize_text_basic(s: str) -> str:
    if pd.isna(s):
        return ""
    out = str(s).strip().upper()
    out = re.sub(r"[“”\"']", "", out)
    out = re.sub(r"\s+", " ", out)
    return out

def key_for_dupes(row):
    est = normalize_text_basic(row.get("ESTABLECIMIENTO", ""))
    dire = normalize_text_basic(row.get("DIRECCION", ""))
    muni = normalize_text_basic(row.get("MUNICIPIO", ""))
    return f"{muni} | {est} | {dire}"

keys = df.apply(key_for_dupes, axis=1)
counts = keys.value_counts()
dupe_keys = counts[counts > 1].head(10)

df_dupes = df.loc[keys.isin(dupe_keys.index), ["CODIGO", "MUNICIPIO", "ESTABLECIMIENTO", "DIRECCION", "JORNADA"]]
print("Posibles duplicados encontrados:", len(df_dupes))
display(df_dupes.head(20))


Teléfonos inválidos encontrados: 1


,CODIGO,ESTABLECIMIENTO,DIRECCION,TELEFONO
70,,,,


Establecimientos con comillas/espacios dobles: 23


,CODIGO,ESTABLECIMIENTO
0,19-01-0078-46,INSTITUTO DIVERSIFICADO ADSCRITO AL INSTITUTO ...
3,19-01-0086-46,"CENTRO EDUCATIVO JUVENIL CATÓLICO ""NUESTRA SEÑ..."
6,19-01-0160-46,"COLEGIO PARTICULAR MIXTO ""LICEO CRISTIANO ZACA..."
10,19-01-0192-46,"COLEGIO ""HIGA"""
11,19-01-0194-46,"COLEGIO ""HIGA"""
16,19-01-0209-46,"CENTRO DE EDUCACIÓN A DISTANCIA ""ITZAMNÁ"""
17,19-01-0211-46,"COLEGIO ""HIGA"""
26,19-01-0942-46,"CENTRO DE EDUCACIÓN A DISTANCIA ""ITZAMNÁ"""
28,19-01-1012-46,"CENTRO EDUCATIVO JUVENIL CATÓLICO ""NUESTRA SEÑ..."
37,19-04-0052-46,"COLEGIO PRIVADO MIXTO ""BETHANIA"""


Direcciones con comillas/espacios dobles: 2


,CODIGO,DIRECCION
0,19-01-0078-46,"7A. AVENIDA ""A"" FINAL Y 9A. CALLE ZONA 2, BARR..."
4,19-01-0116-46,"7A. AVENIDA ""A"" FINAL Y 9A. CALLE ZONA 2 BARRI..."


Posibles duplicados encontrados: 16


,CODIGO,MUNICIPIO,ESTABLECIMIENTO,DIRECCION,JORNADA
3,19-01-0086-46,ZACAPA,"CENTRO EDUCATIVO JUVENIL CATÓLICO ""NUESTRA SEÑ...",12 CALLE A 14-03 ZONA 1,MATUTINA
8,19-01-0175-46,ZACAPA,INSTITUTO TÉCNICO DE EDUCACIÓN INDUSTRIAL,"CIUDAD VICTORIA, BARRIO NUEVO",MATUTINA
10,19-01-0192-46,ZACAPA,"COLEGIO ""HIGA""",BOSQUES DE SAN JULIÁN ZACAPA,VESPERTINA
11,19-01-0194-46,ZACAPA,"COLEGIO ""HIGA""",BOSQUES DE SAN JULIÁN ZACAPA,DOBLE
18,19-01-0213-46,ZACAPA,INSTITUTO TÉCNICO PRIVADO VOCACIONAL,9A. CALLE 17-27 ZONA 3 BARRIO EL TAMARINDAL,SIN JORNADA
21,19-01-0304-46,ZACAPA,INSTITUTO TÉCNICO PRIVADO VOCACIONAL,9A. CALLE 17-27 ZONA 3 BARRIO EL TAMARINDAL,VESPERTINA
22,19-01-0332-46,ZACAPA,INSTITUTO TÉCNICO DE EDUCACIÓN INDUSTRIAL,"CIUDAD VICTORIA, BARRIO NUEVO",DOBLE
28,19-01-1012-46,ZACAPA,"CENTRO EDUCATIVO JUVENIL CATÓLICO ""NUESTRA SEÑ...",12 CALLE A 14-03 ZONA 1,MATUTINA
40,19-04-0071-46,GUALAN,"""LICEO SINAÍ""",BARRIO LAS FLORES,SIN JORNADA
41,19-04-0358-46,GUALAN,"""LICEO SINAÍ""",BARRIO LAS FLORES,VESPERTINA


In [18]:
import pandas as pd
import re

# Cargar el archivo (corrige la ruta según tu sistema operativo)
df = pd.read_csv("salidas_limpias/establecimientos_union_limpio.csv")

# Eliminar NaNs y extraer solo los dígitos numéricos
tel_digits = df['TELEFONO'].fillna("").astype(str).str.replace(r"\D", "", regex=True)

# Definir los dígitos válidos con los que puede comenzar un teléfono en Guatemala
valid_start_digits = ("2", "3", "4", "5", "6", "7")

# Teléfonos inválidos si:
# - No tienen exactamente 8 dígitos
# - O no empiezan con un dígito válido
invalid_mask = (
    (tel_digits.str.len() != 8) |
    (~tel_digits.str.startswith(valid_start_digits))
)

# Mostrar resultados
df_invalid_tel = df.loc[invalid_mask, ["CODIGO", "DEPARTAMENTO", "MUNICIPIO", "ESTABLECIMIENTO", "TELEFONO"]]

print("Teléfonos inválidos encontrados:", len(df_invalid_tel))
display(df_invalid_tel.head(10))


Teléfonos inválidos encontrados: 214


,CODIGO,DEPARTAMENTO,MUNICIPIO,ESTABLECIMIENTO,TELEFONO
83,16-03-0036-46,ALTA VERAPAZ,SAN CRISTOBAL VERAPAZ,INSTITUTO NACIONAL DE EDUCACION DIVERSIFICADA,NaN
110,16-03-1388-46,ALTA VERAPAZ,SAN CRISTOBAL VERAPAZ,COLEGIO MESON,79504027-79504028
136,16-06-2398-46,ALTA VERAPAZ,SAN MIGUEL TUCURU,INSTITUTO PRIVADO MIXTO DE MAGISTERIO BILINGUE...,NaN
151,16-07-0253-46,ALTA VERAPAZ,PANZOS,COLEGIO PRIVADO MIXTO JUAN AMOS COMENIO,4085613
170,16-09-0007-46,ALTA VERAPAZ,SAN PEDRO CARCHA,CENTRO EDUCATIVO NUEVA VIDA,NaN
207,16-09-9259-46,ALTA VERAPAZ,SAN PEDRO CARCHA,COLEGIO STHELLA DE HERNANDEZ,NaN
250,16-14-0054-46,ALTA VERAPAZ,CHAHAL,INSTITUTO NACIONAL DE EDUCACION DIVERSIFICADA,NaN
255,16-15-0002-46,ALTA VERAPAZ,FRAY BARTOLOME DE LAS CASAS,COLEGIO PARTICULAR MIXTO EL EXITO,NaN
300,15-01-0111-46,BAJA VERAPAZ,SALAMA,COLEGIO PARTICULAR MIXTO CIENCIA Y DESARROLLO,79540830-79540909
301,15-01-0112-46,BAJA VERAPAZ,SALAMA,COLEGIO PARTICULAR MIXTO CIENCIA Y DESARROLLO,79540830-79540909


In [9]:
# Helpers de limpieza (texto y teléfono)

import re
import numpy as np
import pandas as pd

def normalize_text_basic(s: str) -> str:
    """Mayúsculas consistentes, sin comillas, espacios colapsados."""
    if pd.isna(s):
        return ""
    out = str(s).strip().upper()
    out = re.sub(r'[“”"“”\'`´]', "", out)     # quitar comillas
    out = re.sub(r"\s+", " ", out)            # colapsar espacios
    return out

def normalize_text_columns(df: pd.DataFrame, cols):
    df2 = df.copy()
    for c in cols:
        if c in df2.columns:
            df2[c] = df2[c].astype(str).apply(normalize_text_basic)
    return df2

def extract_phones(s: str):
    """Devuelve todos los teléfonos de 8 dígitos encontrados en el string."""
    if pd.isna(s) or str(s).strip() == "":
        return []
    digits = re.findall(r"\d+", str(s))
    return [d for d in digits if len(d) == 8]  # formato GT

def normalize_phones_column(df: pd.DataFrame, col_in="TELEFONO"):
    df2 = df.copy()
    if col_in in df2.columns:
        df2[col_in + "_ORIG"] = df2[col_in]
        lst = df2[col_in].apply(extract_phones)
        df2[col_in + "_8DIG"] = lst.apply(lambda L: L[0] if L else "")
        df2[col_in + "_NORMALIZADO"] = lst.apply(lambda L: " | ".join(L) if L else "")
    return df2


In [10]:
#Aplicar limpieza al df cargado

# Normalizar texto en columnas clave
text_cols = [
    "DEPARTAMENTO","MUNICIPIO","ESTABLECIMIENTO","DIRECCION",
    "SUPERVISOR","DIRECTOR","NIVEL","SECTOR","AREA",
    "STATUS","MODALIDAD","JORNADA","PLAN","DEPARTAMENTAL"
]
df_clean = normalize_text_columns(df, text_cols)

# Normalizar teléfonos (conservar original y derivar 8 dígitos / lista)
df_clean = normalize_phones_column(df_clean, "TELEFONO")

# Remover filas totalmente vacías o sin CODIGO (no borramos por contenido)
mask_all_empty = df_clean.replace("", np.nan).isna().all(axis=1)
mask_codigo_empty = df_clean["CODIGO"].astype(str).str.strip().eq("") if "CODIGO" in df_clean.columns else False
df_clean = df_clean.loc[~(mask_all_empty | mask_codigo_empty)].reset_index(drop=True)

# generar una clave normalizada para ayudar a detectar duplicados después de la unión
def key_for_dupes(row):
    est = normalize_text_basic(row.get("ESTABLECIMIENTO",""))
    dire = normalize_text_basic(row.get("DIRECCION",""))
    muni = normalize_text_basic(row.get("MUNICIPIO",""))
    return f"{muni} | {est} | {dire}"

df_clean["CLAVE_NORMALIZADA"] = df_clean.apply(key_for_dupes, axis=1)

# Vista rápida
df_clean.head(10)


,CODIGO,DISTRITO,DEPARTAMENTO,MUNICIPIO,ESTABLECIMIENTO,DIRECCION,TELEFONO,SUPERVISOR,DIRECTOR,NIVEL,...,AREA,STATUS,MODALIDAD,JORNADA,PLAN,DEPARTAMENTAL,TELEFONO_ORIG,TELEFONO_8DIG,TELEFONO_NORMALIZADO,CLAVE_NORMALIZADA
0,19-01-0078-46,19-001,ZACAPA,ZACAPA,INSTITUTO DIVERSIFICADO ADSCRITO AL INSTITUTO ...,"7A. AVENIDA A FINAL Y 9A. CALLE ZONA 2, BARRIO...",79414031,SONIA HAYDEE RUIZ WONG,MARCO ANTONIO LÓPEZ RAMOS,DIVERSIFICADO,...,URBANA,ABIERTA,MONOLINGUE,VESPERTINA,DIARIO(REGULAR),ZACAPA,79414031,79414031,79414031,ZACAPA | INSTITUTO DIVERSIFICADO ADSCRITO AL I...
1,19-01-0079-46,19-001,ZACAPA,ZACAPA,ESCUELA NACIONAL DE CIENCIAS COMERCIALES MIXTA...,4A. CALLE 16-10 ZONA 1. BARRIO EL CALVARIO,57129013,SONIA HAYDEE RUIZ WONG,ELMER OTTONIEL AVALOS MIGUEL,DIVERSIFICADO,...,URBANA,ABIERTA,MONOLINGUE,NOCTURNA,DIARIO(REGULAR),ZACAPA,57129013,57129013,57129013,ZACAPA | ESCUELA NACIONAL DE CIENCIAS COMERCIA...
2,19-01-0084-46,19-001,ZACAPA,ZACAPA,INSTITUTO ADOLFO V. HALL DE ORIENTE,BARRIO CRUZ DE MAYO ZONA 3,39913254,SONIA HAYDEE RUIZ WONG,RAÚL ANIBAL PINEDA PEÑATE,DIVERSIFICADO,...,URBANA,ABIERTA,MONOLINGUE,DOBLE,DIARIO(REGULAR),ZACAPA,39913254,39913254,39913254,ZACAPA | INSTITUTO ADOLFO V. HALL DE ORIENTE |...
3,19-01-0086-46,19-019,ZACAPA,ZACAPA,CENTRO EDUCATIVO JUVENIL CATÓLICO NUESTRA SEÑO...,12 CALLE A 14-03 ZONA 1,79410382,HUGO WILFREDO VARGAS CHACON,FIDIAS ROBERTO MONROY LEMUS,DIVERSIFICADO,...,URBANA,ABIERTA,MONOLINGUE,MATUTINA,DIARIO(REGULAR),ZACAPA,79410382,79410382,79410382,ZACAPA | CENTRO EDUCATIVO JUVENIL CATÓLICO NUE...
4,19-01-0116-46,19-001,ZACAPA,ZACAPA,"INSTITUTO NACIONAL DE EDUCACION DIVERSIFICADA,...",7A. AVENIDA A FINAL Y 9A. CALLE ZONA 2 BARRIO ...,42053474,SONIA HAYDEE RUIZ WONG,MARCY JULISSA JIATZ ALONZO,DIVERSIFICADO,...,URBANA,ABIERTA,MONOLINGUE,VESPERTINA,DIARIO(REGULAR),ZACAPA,42053474,42053474,42053474,ZACAPA | INSTITUTO NACIONAL DE EDUCACION DIVER...
5,19-01-0149-46,19-019,ZACAPA,ZACAPA,COLEGIO BILINGÜE MONTESSORI,4TA. CALLE 3-41 ZONA 2,79410965,HUGO WILFREDO VARGAS CHACON,MAGDA YOHANA SÁNCHEZ MONTERROSO,DIVERSIFICADO,...,URBANA,ABIERTA,MONOLINGUE,MATUTINA,DIARIO(REGULAR),ZACAPA,79410965,79410965,79410965,ZACAPA | COLEGIO BILINGÜE MONTESSORI | 4TA. CA...
6,19-01-0160-46,19-019,ZACAPA,ZACAPA,COLEGIO PARTICULAR MIXTO LICEO CRISTIANO ZACAP...,12 AVENIDA Y 8A. CALLE ESQUINA ZONA 1,79410316,HUGO WILFREDO VARGAS CHACON,LUCY RAQUEL APARICIO ROLDÁN,DIVERSIFICADO,...,URBANA,ABIERTA,MONOLINGUE,MATUTINA,DIARIO(REGULAR),ZACAPA,79410316,79410316,79410316,ZACAPA | COLEGIO PARTICULAR MIXTO LICEO CRISTI...
7,19-01-0170-46,19-019,ZACAPA,ZACAPA,COLEGIO CRISTIANO VERBO ZACAPA,CALZADA RAMIRO DE LEÓN CARPIO A UN COSTADO DEL...,58289369,HUGO WILFREDO VARGAS CHACON,LIDIA ISABEL PÉREZ VILLEDA,DIVERSIFICADO,...,URBANA,ABIERTA,MONOLINGUE,MATUTINA,DIARIO(REGULAR),ZACAPA,58289369,58289369,58289369,ZACAPA | COLEGIO CRISTIANO VERBO ZACAPA | CALZ...
8,19-01-0175-46,19-019,ZACAPA,ZACAPA,INSTITUTO TÉCNICO DE EDUCACIÓN INDUSTRIAL,"CIUDAD VICTORIA, BARRIO NUEVO",79415915,HUGO WILFREDO VARGAS CHACON,ALEJANDRA MARÍA PALACIOS GARCÍA,DIVERSIFICADO,...,URBANA,ABIERTA,MONOLINGUE,MATUTINA,DIARIO(REGULAR),ZACAPA,79415915,79415915,79415915,ZACAPA | INSTITUTO TÉCNICO DE EDUCACIÓN INDUST...
9,19-01-0183-46,19-019,ZACAPA,ZACAPA,LICEO CRISTO REY,8A. CALLE 9-06 ZONA 2 BARRIO LA REFORMA,54138046,HUGO WILFREDO VARGAS CHACON,EDNA EDITH RAMÍREZ CORDÓN,DIVERSIFICADO,...,URBANA,ABIERTA,MONOLINGUE,DOBLE,FIN DE SEMANA,ZACAPA,54138046,54138046,54138046,ZACAPA | LICEO CRISTO REY | 8A. CALLE 9-06 ZON...


In [11]:
# Limpieza masiva de todos los CSV

for name, path in files.items():
    # 1) Cargar y estandarizar encabezados
    dfi = load_and_fix_header(path)
    dfi.columns = standardize_colnames(dfi.columns)
    
    # Normalizar textos
    text_cols = [
        "DEPARTAMENTO","MUNICIPIO","ESTABLECIMIENTO","DIRECCION",
        "SUPERVISOR","DIRECTOR","NIVEL","SECTOR","AREA",
        "STATUS","MODALIDAD","JORNADA","PLAN","DEPARTAMENTAL"
    ]
    dfi = normalize_text_columns(dfi, text_cols)
    
    # Normalizar teléfonos
    dfi = normalize_phones_column(dfi, "TELEFONO")
    
    # Eliminar filas vacías o sin CODIGO
    mask_all_empty = dfi.replace("", np.nan).isna().all(axis=1)
    mask_codigo_empty = dfi["CODIGO"].astype(str).str.strip().eq("") if "CODIGO" in dfi.columns else False
    dfi = dfi.loc[~(mask_all_empty | mask_codigo_empty)].reset_index(drop=True)
    
    # Guardar CSV limpio
    out_path = out_dir / f"{name}_limpio.csv"
    dfi.to_csv(out_path, index=False, encoding="utf-8")
    print(f"Guardado: {out_path}")


Guardado: salidas_limpias\altaverapaz_limpio.csv
Guardado: salidas_limpias\bajaverapaz_limpio.csv
Guardado: salidas_limpias\chimaltenango_limpio.csv
Guardado: salidas_limpias\chiquimula_limpio.csv
Guardado: salidas_limpias\ciudadcapital_limpio.csv
Guardado: salidas_limpias\elprogreso_limpio.csv
Guardado: salidas_limpias\escuintla_limpio.csv
Guardado: salidas_limpias\guatemala_limpio.csv
Guardado: salidas_limpias\izabal_limpio.csv
Guardado: salidas_limpias\jalapa_limpio.csv
Guardado: salidas_limpias\jutiapa_limpio.csv
Guardado: salidas_limpias\peten_limpio.csv
Guardado: salidas_limpias\quetzaltenango_limpio.csv
Guardado: salidas_limpias\quiche_limpio.csv
Guardado: salidas_limpias\retalhuleu_limpio.csv
Guardado: salidas_limpias\sacatepequez_limpio.csv
Guardado: salidas_limpias\sanmarcos_limpio.csv
Guardado: salidas_limpias\santarosa_limpio.csv
Guardado: salidas_limpias\solola_limpio.csv
Guardado: salidas_limpias\suchitipequez_limpio.csv
Guardado: salidas_limpias\totonicapan_limpio.csv
Gu

In [12]:
import glob

# Leer todos los *_limpio.csv
all_clean = []
for f in glob.glob(str(out_dir / "*_limpio.csv")):
    all_clean.append(pd.read_csv(f, dtype=str, keep_default_na=False))

df_union = pd.concat(all_clean, ignore_index=True)

print("Unión — forma:", df_union.shape)
print("Duplicados por CODIGO:", int(df_union["CODIGO"].duplicated().sum()))

# si no existe, generamos CLAVE_NORMALIZADA para apoyo a duplicados
def normalize_text_basic(s: str) -> str:
    if pd.isna(s): return ""
    out = str(s).strip().upper()
    out = re.sub(r'[“”"“”\'`´]', "", out)
    out = re.sub(r"\s+", " ", out)
    return out

def key_for_dupes(row):
    est = normalize_text_basic(row.get("ESTABLECIMIENTO",""))
    dire = normalize_text_basic(row.get("DIRECCION",""))
    muni = normalize_text_basic(row.get("MUNICIPIO",""))
    return f"{muni} | {est} | {dire}"

if "CLAVE_NORMALIZADA" not in df_union.columns:
    df_union["CLAVE_NORMALIZADA"] = df_union.apply(key_for_dupes, axis=1)

union_path = out_dir / "establecimientos_union_limpio.csv"
df_union.to_csv(union_path, index=False, encoding="utf-8")
print("Guardado:", union_path.resolve())


Unión — forma: (12608, 21)
Duplicados por CODIGO: 6304
Guardado: C:\Users\Silvia\Documents\Cuarto Año\Segundo Semestre\Data\Proyecto1-DataScience-LimpiezaDeDatos\salidas_limpias\establecimientos_union_limpio.csv


In [15]:
import pandas as pd

# Cargar el archivo unificado
df = pd.read_csv(r"salidas_limpias\establecimientos_union_limpio.csv")

# Validar columna CODIGO 
print("Duplicados por CODIGO:", df["CODIGO"].duplicated().sum())
print("Valores nulos en CODIGO:", df["CODIGO"].isna().sum())

# Limpiar espacios y asegurar que sea string
df["CODIGO"] = df["CODIGO"].astype(str).str.strip()

# Validar columnas de texto
cols_texto = ["DEPARTAMENTO", "MUNICIPIO", "SECTOR", "AREA"]

for col in cols_texto:
    print(f"\nColumna: {col}")
    print("Valores únicos antes de limpieza:", df[col].nunique())
    print("Ejemplos:", df[col].unique()[:10])  # primeras 10 categorías

    # Limpiar espacios, poner en título, quitar dobles espacios
    df[col] = df[col].astype(str).str.strip().str.title().str.replace(r'\s+', ' ', regex=True)

# 3. Validar TELEFONO 
print("\nValores únicos TELEFONO (antes de limpieza):", df["TELEFONO"].nunique())
df["TELEFONO"] = df["TELEFONO"].astype(str).str.replace(r'\D+', '', regex=True)  # dejar solo dígitos

#Longitudes de teléfono
df["long_tel"] = df["TELEFONO"].apply(len)
print("Distribución longitudes de teléfono:")
print(df["long_tel"].value_counts())

# Guardar la versión validada preliminar
df.drop(columns="long_tel").to_csv("salidas_limpias/salidas_validado.csv", index=False)
print("\nGuardado: salidas_limpias/salidas_validado.csv")


Duplicados por CODIGO: 6304
Valores nulos en CODIGO: 0

Columna: DEPARTAMENTO
Valores únicos antes de limpieza: 22
Ejemplos: ['ALTA VERAPAZ' 'BAJA VERAPAZ' 'CHIMALTENANGO' 'CHIQUIMULA'
 'CIUDAD CAPITAL' 'EL PROGRESO' 'ESCUINTLA' 'GUATEMALA' 'IZABAL' 'JALAPA']

Columna: MUNICIPIO
Valores únicos antes de limpieza: 315
Ejemplos: ['COBAN' 'SANTA CRUZ VERAPAZ' 'SAN CRISTOBAL VERAPAZ' 'TACTIC' 'TAMAHU'
 'SAN MIGUEL TUCURU' 'PANZOS' 'SENAHU' 'SAN PEDRO CARCHA'
 'SAN JUAN CHAMELCO']

Columna: SECTOR
Valores únicos antes de limpieza: 4
Ejemplos: ['PRIVADO' 'OFICIAL' 'MUNICIPAL' 'COOPERATIVA']

Columna: AREA
Valores únicos antes de limpieza: 3
Ejemplos: ['URBANA' 'RURAL' 'SIN ESPECIFICAR']

Valores únicos TELEFONO (antes de limpieza): 4013
Distribución longitudes de teléfono:
long_tel
8     12398
0        92
16       70
7        34
6         6
24        2
4         2
2         2
15        2
Name: count, dtype: int64

Guardado: salidas_limpias/salidas_validado.csv


In [17]:
import pandas as pd
import numpy as np

df = pd.read_csv(r"salidas_limpias\establecimientos_union_limpio.csv", dtype=str, keep_default_na=False)

# Limpia espacios
df["CODIGO"] = df["CODIGO"].astype(str).str.strip()

print("Antes:")
print("- Duplicados por CODIGO:", df["CODIGO"].duplicated().sum())
print("- Filas con CODIGO vacío:", (df["CODIGO"]=="").sum())

# Quitar filas 100% vacías
mask_all_empty = df.replace("", np.nan).isna().all(axis=1)
df = df.loc[~mask_all_empty].copy()

# Quitar filas con CODIGO vacío (no se puede identificar)
df = df.loc[df["CODIGO"]!=""].copy()

# Quitar duplicados EXACTOS de fila
before = len(df)
df = df.drop_duplicates()
print(f"- Se eliminaron {before - len(df)} duplicados EXACTOS de fila")

# Analizar duplicados por CODIGO (mismo código en 2+ filas)
dupe_mask = df["CODIGO"].duplicated(keep=False)
dupes = df.loc[dupe_mask].sort_values(["CODIGO","JORNADA","ESTABLECIMIENTO"])

print("- Registros con CODIGO repetido (después de exact dedup):", dupes.shape[0])

# Si los duplicados por CODIGO son porque el mismo establecimiento opera en varias JORNADAS, se conservan.
# Para que quede claro en el dataset final, marcamos cuántos registros tiene cada CODIGO.
cnt = df["CODIGO"].value_counts()
df["REGS_CON_MISMO_CODIGO"] = df["CODIGO"].map(cnt)

df.shape


Antes:
- Duplicados por CODIGO: 6304
- Filas con CODIGO vacío: 0
- Se eliminaron 0 duplicados EXACTOS de fila
- Registros con CODIGO repetido (después de exact dedup): 12608


(12608, 22)

In [ ]:
import re
import pandas as pd

def norm_text_title(s: str) -> str:
    if pd.isna(s) or s == "": return ""
    s = str(s).strip()
    s = re.sub(r"\s+", " ", s)   # colapsar espacios
    return s.title()

# Texto base: nombres y direcciones en Título
for col in ["DEPARTAMENTO","MUNICIPIO","ESTABLECIMIENTO","DIRECCION","SUPERVISOR","DIRECTOR"]:
    if col in df.columns:
        df[col] = df[col].astype(str).apply(norm_text_title)

# SECTOR -> valores controlados
map_sector = {
    "Oficial":"OFICIAL","Privado":"PRIVADO","Municipal":"MUNICIPAL","Cooperativa":"COOPERATIVA"
}
if "SECTOR" in df.columns:
    df["SECTOR"] = df["SECTOR"].astype(str).str.strip().str.title().map(map_sector).fillna(df["SECTOR"].astype(str).str.upper())

# AREA -> valores controlados
map_area = {"Urbana":"URBANA","Rural":"RURAL","Sin Especificar":"SIN ESPECIFICAR"}
if "AREA" in df.columns:
    df["AREA"] = df["AREA"].astype(str).str.strip().str.title().map(map_area).fillna(df["AREA"].astype(str).str.upper())

# JORNADA -> valores controlados (amplía si ves otras variantes)
map_jornada = {
    "Matutina":"MATUTINA","Vespertina":"VESPERTINA","Doble":"DOBLE",
    "Sin Jornada":"SIN JORNADA","Nocturna":"NOCTURNA","Sabatina":"SÁBATINA"
}
if "JORNADA" in df.columns:
    df["JORNADA"] = df["JORNADA"].astype(str).str.strip().str.title().map(map_jornada).fillna(df["JORNADA"].astype(str).str.upper())

# Chequeo rápido
print("SECTOR:", sorted(df["SECTOR"].dropna().unique())[:10])
print("AREA:", sorted(df["AREA"].dropna().unique()))
print("JORNADA:", sorted(df["JORNADA"].dropna().unique())[:10])


SECTOR: ['COOPERATIVA', 'MUNICIPAL', 'OFICIAL', 'PRIVADO']
AREA: ['RURAL', 'SIN ESPECIFICAR', 'URBANA']
JORNADA: ['DOBLE', 'INTERMEDIA', 'MATUTINA', 'NOCTURNA', 'SIN JORNADA', 'VESPERTINA']


In [ ]:
import re
import pandas as pd

def norm_text_title(s: str) -> str:
    if pd.isna(s) or s == "": return ""
    s = str(s).strip()
    s = re.sub(r"\s+", " ", s)   # colapsar espacios
    return s.title()

# Texto base: nombres y direcciones en Título
for col in ["DEPARTAMENTO","MUNICIPIO","ESTABLECIMIENTO","DIRECCION","SUPERVISOR","DIRECTOR"]:
    if col in df.columns:
        df[col] = df[col].astype(str).apply(norm_text_title)

# SECTOR -> valores controlados
map_sector = {
    "Oficial":"OFICIAL","Privado":"PRIVADO","Municipal":"MUNICIPAL","Cooperativa":"COOPERATIVA"
}
if "SECTOR" in df.columns:
    df["SECTOR"] = df["SECTOR"].astype(str).str.strip().str.title().map(map_sector).fillna(df["SECTOR"].astype(str).str.upper())

# AREA -> valores controlados
map_area = {"Urbana":"URBANA","Rural":"RURAL","Sin Especificar":"SIN ESPECIFICAR"}
if "AREA" in df.columns:
    df["AREA"] = df["AREA"].astype(str).str.strip().str.title().map(map_area).fillna(df["AREA"].astype(str).str.upper())

# JORNADA -> valores controlados (amplía si ves otras variantes)
map_jornada = {
    "Matutina":"MATUTINA","Vespertina":"VESPERTINA","Doble":"DOBLE",
    "Sin Jornada":"SIN JORNADA","Nocturna":"NOCTURNA","Sabatina":"SÁBATINA"
}
if "JORNADA" in df.columns:
    df["JORNADA"] = df["JORNADA"].astype(str).str.strip().str.title().map(map_jornada).fillna(df["JORNADA"].astype(str).str.upper())

# Chequeo rápido
print("SECTOR:", sorted(df["SECTOR"].dropna().unique())[:10])
print("AREA:", sorted(df["AREA"].dropna().unique()))
print("JORNADA:", sorted(df["JORNADA"].dropna().unique())[:10])


SECTOR: ['COOPERATIVA', 'MUNICIPAL', 'OFICIAL', 'PRIVADO']
AREA: ['RURAL', 'SIN ESPECIFICAR', 'URBANA']
JORNADA: ['DOBLE', 'INTERMEDIA', 'MATUTINA', 'NOCTURNA', 'SIN JORNADA', 'VESPERTINA']


In [ ]:
def only_digits(s: str) -> str:
    return re.sub(r"\D+", "", s or "")

if "TELEFONO" in df.columns:
    # conservar original
    df["TELEFONO_ORIG"] = df["TELEFONO"]
    # solo dígitos
    df["TELEFONO_DIG"] = df["TELEFONO"].astype(str).apply(only_digits)

    # extraer TODOS los de 8 dígitos que vengan en la cadena
    def extract_8_list(s):
        digs = re.findall(r"\d+", s or "")
        return [d for d in digs if len(d)==8]

    tel_lists = df["TELEFONO"].astype(str).apply(extract_8_list)
    df["TELEFONO_8DIG"] = tel_lists.apply(lambda L: L[0] if L else "")
    df["TELEFONOS_VALIDOS"] = tel_lists.apply(lambda L: " | ".join(L))

    # resumen
    print("Longitudes TELEFONO_DIG:", df["TELEFONO_DIG"].str.len().value_counts().sort_index().to_dict())
    print("Vacíos en TELEFONO_8DIG:", (df["TELEFONO_8DIG"]=="").sum())


Longitudes TELEFONO_DIG: {0: 46, 2: 2, 5: 1, 6: 2, 7: 9, 8: 2995, 9: 3212, 15: 1, 16: 32, 17: 3, 24: 1}
Vacíos en TELEFONO_8DIG: 72


In [ ]:
out_final = "establecimientos_validado_final.csv"
df.to_csv(out_final, index=False, encoding="utf-8")
print("Guardado:", out_final, "| Forma:", df.shape)


Guardado: establecimientos_validado_final.csv | Forma: (6304, 22)
